
### How to use a logistic classification regression model to predict Churn on pre-processed Google Analytics 4 data using  BigQueryML

[original source](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/bigquery_ml/bqml-online-prediction.ipynb)

### Objective

In this tutorial, you learn how to train and deploy a churn prediction model for real-time inference, with the data in BigQuery and model trained using BigQuery ML, registered to Vertex AI Model Registry, and deployed to an endpoint on Vertex AI for online predictions.

This tutorial uses the following Google Cloud data analytics and ML services:

- BigQuery
- BigQuery ML
- Vertex AI Model Registry
- Vertex endpoints


The steps performed include:

- Using Python & SQL to query the public data in BigQuery
- Preparing the data for modeling
- Training a classification model using BigQuery ML and registering it to Vertex AI Model Registry
- Inspecting the model on Vertex AI Model Registry
- Deploying the model to an endpoint on Vertex AI
- Making sample online predictions to the model endpoint


### Dataset

The dataset, <a href="https://console.cloud.google.com/bigquery?project=bigquery-public-data&d=ga4_obfuscated_sample_ecommerce&p=bigquery-public-data&page=dataset" target="_blank">available publicly on BigQuery</a>, comes from obfuscated <a href="https://support.google.com/analytics/answer/10937659" target="_blank">Google Analytics 4 data</a> from the <a href="https://shop.googlemerchandisestore.com/" target="_blank">Google Merchandise Store</a>).

### Install additional packages

Install the following packages required to execute this notebook. 

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q google-cloud-bigquery db-dtypes

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Project Variables 

In [ ]:
# Project variables 
#
# These are the project variable used in this ML Model: 
#

PROJECT_ID = "" # @param {type:"string"}
bqml_type = "log_reg" # @param {type:"string"}
BQML_MODEL_NAME = "bqml_log_reg_analytics_churn_predict_model"
job_display_name = BQML_MODEL_NAME + "_job"
ENDPOINT_NAME = BQML_MODEL_NAME + "_endpoint"

# datasets
BQ_DATASET_NAME = BQML_MODEL_NAME + "_dataset"
sql_create_dataset = f"""CREATE SCHEMA IF NOT EXISTS {BQ_DATASET_NAME}"""

# bucket details
BUCKET_NAME = "bqml_tutorials"
BUCKET_URI = f"gs://{BUCKET_NAME}/{bqml_type}/"
OUTPUTBUCKET = f"gs://bqml_datasets_predictions/{bqml_type}/"

# Region 
REGION = "us-central1" # @param {type: "string"} 

Otherwise, set your project ID here.

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You might not be able to use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about <a href="https://cloud.google.com/vertex-ai/docs/general/locations" target="_blank">Vertex AI regions</a>.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

In [ ]:
SERVICE_ACCOUNT = ""  # @param {type:"string"}
print(SERVICE_ACCOUNT)

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

### Import libraries

In [ ]:
from typing import Union

import google.cloud.aiplatform as vertex_ai
import pandas as pd
from google.cloud import bigquery

### Initialize Vertex AI and BigQuery SDKs for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
vertex_ai.init(project=PROJECT_ID, location=REGION)

Create the BigQuery client.

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID)

Use a helper function for sending queries to BigQuery.

In [ ]:
# Wrapper to use BigQuery client to run query/job, return job ID or result as DF
def run_bq_query(sql: str) -> Union[str, pd.DataFrame]:
    """
    Input: SQL query, as a string, to execute in BigQuery
    Returns the query results as a pandas DataFrame, or error, if any
    """

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

## BigQuery ML Model Training & Validation

BigQuery ML (BQML) provides the capability to train ML tabular models, such as classification, regression, forecasting, and matrix factorization, in BigQuery using SQL syntax directly. BigQuery ML uses the scalable infrastructure of BigQuery ML so you don't need to set up additional infrastructure for training or batch serving.

In [ ]:
sql_create_dataset = f"""CREATE SCHEMA IF NOT EXISTS {BQ_DATASET_NAME}"""

print(sql_create_dataset)

run_bq_query(sql_create_dataset)

### Using logistic regression model for classification on pre-processed Google Analytics 4 data

Inpect data that has been pre-processed from <a href="https://support.google.com/analytics/answer/10937659" target="_blank">Google Analytics 4 data from the Google Merchandise Store</a> so that it can be used for classification. For more information on how this data was prepared, read <a href="https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml" target="_blank">this blog post</a>.

As seen below, each row represents a single user, and the columns represent their demographic features, their aggregated behavioral features in the first 24 hours of visiting the Google Merchandise Store, and the label (whether the user churned or returned any time after the first 24 hours).

In [ ]:
sql_inspect = """
SELECT
    *
FROM
    `bqmlpublic.demo_ga4churnprediction.training_data`
LIMIT
    100
"""
run_bq_query(sql_inspect)

### Train a classification model using BigQuery ML

The query below trains a logistic regression model using BigQuery ML. BigQuery resources are used to train the model.

In the `OPTIONS` parameter:
* with `model_registry="vertex_ai"`, the BigQuery ML model will automatically be <a href="https://cloud.google.com/vertex-ai/docs/model-registry/model-registry-bqml" target="_blank">registered to Vertex AI Model Registry</a>, which enables you to view all of your registered models and its versions on Google Cloud in one place.

* `vertex_ai_model_version_aliases allows you to set aliases to help you keep track of your model version (<a href="https://cloud.google.com/vertex-ai/docs/model-registry/model-alias" target="_blank">documentation</a>).

In [ ]:
# this cell may take ~1 min to run

sql_train_model_bqml = f"""
CREATE OR REPLACE MODEL {BQ_DATASET_NAME}.{BQML_MODEL_NAME}    
OPTIONS(
  MODEL_TYPE="LOGISTIC_REG",
  input_label_cols=["churned"],
  model_registry="vertex_ai",
  vertex_ai_model_version_aliases=['logistic_reg', 'experimental']
) AS

SELECT
  * EXCEPT(user_first_engagement, user_pseudo_id)
FROM
  bqmlpublic.demo_ga4churnprediction.training_data
"""

print(sql_train_model_bqml)

run_bq_query(sql_train_model_bqml)

### Model evaluation

With the model created, you can now evaluate the logistic regression model. Behind the scenes, BigQuery ML automatically <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create#data_split_method" target="_blank">split the data</a>, which makes it easier to quickly train and evaluate models.

In [ ]:
sql_evaluate_model = f"""
SELECT
  *
FROM
  ML.EVALUATE(MODEL {BQ_DATASET_NAME}.{BQML_MODEL_NAME})
"""

print(sql_evaluate_model)

run_bq_query(sql_evaluate_model)

These metrics help you understand the performance of the model. 

There are various metrics for logistic regression and other model types (full list of metrics can be found in the <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate#mlevaluate_output" target="_blank">documentation</a>).

### Batch prediction (with Explainable AI)

Make a batch prediction in BigQuery ML on the original training data to check the probability of churn for each of the users, as seen in the `probability` column, with the predicted label under the `predicted_churn` column.

<a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-explain-predict" target="_blank">ML.EXPLAIN_PREDICT</a> has built-in <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-xai-overview" target="_blank">Explainable AI</a>. This allows you to see the top contributing features to each prediction and interpret how it was computed.

In [ ]:
sql_explain_predict = f"""
SELECT
  *
FROM
  ML.EXPLAIN_PREDICT(MODEL {BQ_DATASET_NAME}.{BQML_MODEL_NAME},
    (SELECT * FROM bqmlpublic.demo_ga4churnprediction.training_data LIMIT 100)
    )
"""

print(sql_explain_predict)

run_bq_query(sql_explain_predict)

Since the `top_feature_attributions` is a nested column, you can unnest the array (<a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/arrays" target="_blank">documentation</a>) into separate rows for each of the features. In other words, since ML.EXPLAIN_PREDICT provides the top 5 most important features, using `UNNEST` results in 5 rows per prediction:

In [ ]:
sql_explain_predict = f"""
SELECT
  tfa.*,
  predicted_churned,
  probability,
  baseline_prediction_value,
  prediction_value,
  approximation_error,
  user_pseudo_id
FROM
  ML.EXPLAIN_PREDICT(MODEL {BQ_DATASET_NAME}.{BQML_MODEL_NAME},
    (SELECT * FROM bqmlpublic.demo_ga4churnprediction.training_data LIMIT 100)
    ),
  UNNEST(top_feature_attributions) as tfa
WHERE
  user_pseudo_id = "7666337.2408476627"
"""

print(sql_explain_predict)

run_bq_query(sql_explain_predict)

### Inspect the model on Vertex AI Model Registry

When the model was trained in BigQuery ML, the line `model_registry="vertex_ai"` registered the model to Vertex AI Model Registry automatically upon completion.

You can view the model on the <a href="https://console.cloud.google.com/vertex-ai/models" target="_blank">Vertex AI Model Registry page</a>, or use the code below to check that it was successfully registered:

In [ ]:
model = vertex_ai.Model(model_name=BQML_MODEL_NAME)

print(model.gca_resource)

### Deploy the model to an endpoint

While BigQuery ML supports batch prediction with <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict" target="_blank">ML.PREDICT</a> and <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-explain-predict" target="_blank">ML.EXPLAIN_PREDICT</a>, BigQuery ML is not suitable for real-time predictions where you need low latency predictions with potentially high frequency of requests.

In other words, deploying the BigQuery ML model to an endpoint enables you to do online predictions.

#### Create a Vertex AI endpoint

To deploy your model to an endpoint, you will first need to create an endpoint before you deploy the model to it.

In [ ]:

endpoint = vertex_ai.Endpoint.create(
    display_name=ENDPOINT_NAME,
    project=PROJECT_ID,
    location=REGION,
)

print(endpoint.display_name)
print(endpoint.resource_name)

#### List endpoints

List the endpoints to make sure it has successfully been created. (You can also view your endpoints on the <a href="https://console.cloud.google.com/vertex-ai/endpoints" target="_blank">Vertex AI Endpoints page</a>).

In [ ]:
endpoint.list()

#### Deploy model to Vertex endpoint

With the new endpoint, you can now deploy your model.

In [ ]:
# deploying the model to the endpoint may take 10-15 minutes
model.deploy(endpoint=endpoint)

You can also check on the status of your model by visiting the <a href="https://console.cloud.google.com/vertex-ai/endpoints" target="_blank">Vertex AI Endpoints page</a>.

### Make online predictions to the endpoint

Using a sample of the training data, you can test the endpoint to make online predictions.

In [ ]:
df_sample_requests_list = [
    {
        "country": "Turkey",
        "operating_system": "Web",
        "language": "None",
        "cnt_user_engagement": 28,
        "cnt_page_view": 37,
        "cnt_view_item": 6,
        "cnt_view_promotion": 15,
        "cnt_select_promotion": 4,
        "cnt_add_to_cart": 0,
        "cnt_begin_checkout": 0,
        "cnt_add_shipping_info": 0,
        "cnt_add_payment_info": 0,
        "cnt_purchase": 0,
        "month": 1,
        "julianday": 1,
        "dayofweek": 6,
    },
    {
        "country": "Macao",
        "operating_system": "Web",
        "language": "None",
        "cnt_user_engagement": 2,
        "cnt_page_view": 4,
        "cnt_view_item": 0,
        "cnt_view_promotion": 0,
        "cnt_select_promotion": 0,
        "cnt_add_to_cart": 0,
        "cnt_begin_checkout": 0,
        "cnt_add_shipping_info": 0,
        "cnt_add_payment_info": 0,
        "cnt_purchase": 0,
        "month": 1,
        "julianday": 16,
        "dayofweek": 7,
    },
]

In [ ]:
prediction = endpoint.predict(df_sample_requests_list)
print(prediction)

You can then extract the predictions from the prediction response

In [ ]:
prediction.predictions

## Cleaning up

To clean up all Google Cloud resources used in this project, you can <a href="https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects" target="_blank">delete the Google Cloud
project</a> you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Undeploy model from endpoint and delete endpoint
endpoint.undeploy_all()
endpoint.delete()

# Delete BigQuery dataset, including the BigQuery ML model
! bq rm -r -f $PROJECT_ID:$BQ_DATASET_NAME